## In this Notebook
* examples of applications of ADI 
    - Modelling electrical networks
    - Spectral discretizations of PDEs
    - image denoising

In [3]:
using Random
using LinearAlgebra
using SparseArrays
using Plots
using Statistics

include("./randadi.jl")
# using .RandAdi

Main.RandAdi

### Dynamical Systems
Given $x,u,y$ as state, input, and output vectors, we can work with the simple discrete-time linear system representing

$$\dot{x}=A x+B u, \quad y=C x$$

One application might be to model a complex electronic circuit. For this toy example (see references), consider an RC circuit with $n$ different voltage inputs and a conductance matrix $G \in \mathbb R ^ { n \times n}$. Note that the conductance matrix is constructed with $\frac{1}{r_{ij}}$ for the resistances of each component in a grid $r_{ij}$. 

![](rccirc.png)

We have $c_{k} \dot{v}_{k}=-i_{k}$ for each input, as well as $i = Gv$ for the current in the whole circuit. We represent the capacitances with $C=\operatorname{diag}\left(c_{1}, \ldots, c_{n}\right)$, resulting in the system

$$\dot{v}=-C^{-1} G v$$

If we input a signal at $v_1$ for example, the effects on the sytem can be modelled. This is represented below in the `gen_circ` function.

One application is modelling Very-large-scale integration (VLSI) circuits. These chips operate on a high enough frequency, and are dense enough with microscopic components, that the distance between transistors needs to be modelled.

The Sylvester equation arises in the Sylvester controller equation, which is included in the Control Theory section below.

In [4]:
# Creats an incident matrix for a graph 
# structured as a n x m lattice
function incident_matrix(n,m)
    # for now
    numnodes = n * m
    
    verts = zeros(numnodes, m*(n-1))
    horz = zeros(numnodes, n* (m-1))
    
    # vertical edges in grid
    for e = 1:(m*(n-1))
        q = div((e-1), (n-1))
        r = (e-1) % (n-1)
        verts[n * q + r + 1,e] = 1
        verts[n * q + r + 2,e] = -1
    end
    
    # horizontal edges in grid
    for e = 1:(n*(m-1))
        q = div(e, n)
        r = (e) % (n)
        horz[n * q + r ,e] = 1
        horz[n * q + r + n,e] = -1
    end
    
    Ag = [verts horz]
end

incident_matrix (generic function with 1 method)

In [5]:
# Generates the matrices plugged into the Sylvester Equation
function gen_circ(n)
    # setup from VLSI notes
    r = Int(sqrt(n))
    c = Int(sqrt(n))
    Ag = incident_matrix(r,c)
    Ac = incident_matrix(r,c)

    numnodes = r * c
    numedges = r * (c - 1) + c * (r - 1)

    Gc = Diagonal(rand(numedges))
    d =  numnodes + 2 * numedges

    G = zeros(d, d)
    G[1:numnodes,1:numnodes] = Ag * Gc * Ag'
    G[numnodes+1:end,1:numnodes]  = [Ag Ag]'
    G[1:numnodes, numnodes + 1:end] = [Ag Ag]''

    Cinv = Diagonal(1 ./ rand(numedges))
    G = Cinv * G
    
    B = Diagonal(rand(numedges))

    C = zeros(d, d)
    C[1:numnodes,1:numnodes] = Ac * Cc * Ac'
    C[numnodes + 1: numnodes + numedges,numnodes + 1: numnodes + numedges]  = -1 .* Lc
    
    # matrices for sylvester equation
    Gi = inv(G) # fix this later; how to represent inverse of incidence matrix?
    
    return Gi * C, Gi * B
end

gen_circ (generic function with 1 method)

In [ ]:
Lc = [1,2,3,4,5,6]
Dinv = Diagonal(1 ./ Lc)
D = Diagonal(Lc)
P = rand(3,6)

A = P * D * P'
B = P * Dinv * P'
# C = A * B

### cells testing the VLSI representation equation
- problem: different form from x = Mx' + Bu.
- To convert to that form we need to invert the matrix C from Gx + Cx' = Bu

In [ ]:
n = 4

r = Int(sqrt(n))
c = Int(sqrt(n))
Ag = incident_matrix(r,c)
Ac = incident_matrix(r,c)

numnodes = r * c
numedges = r * (c - 1) + c * (r - 1)
# print(numedges)

Gc = Diagonal(rand(numedges))
Cc = Diagonal(rand(numedges))
Lc = Diagonal(rand(numedges))
d =  numnodes + 2 * numedges

In [ ]:
G = zeros(d, d)
G[1:numnodes,1:numnodes] = Ag * Gc * Ag'
G[numnodes+1:end,1:numnodes]  = [Ag Ag]'
G[1:numnodes, numnodes + 1:end] = [Ag Ag]''

C = zeros(d, d)
C[1:numnodes,1:numnodes] = Ac * Cc * Ac'
C[numnodes + 1: numnodes + numedges,numnodes + 1: numnodes + numedges]  = -1 .* Lc

# matrices for sylvester equation
Cinv = inv(C)# fix this later; how to represent inverse of incidence matrix?

B = Diagonal(rand(numedges))

return -1 * Cinv * G, Cinv * B

In [ ]:
M,B= gen_circ(9)
Cinv = Diagonal(1 ./ rand(numedges))
G = Cinv * G
display("text/plain",G)
display("text/plain",C)

The Sylvester equation is instead of the form  $XA - TX = GC$. We are given $A$ (dense, nonnormal) and $C$, and can construct T and G. By choosing T with disjoint spectrum from A we have better guarantees on the number of iterations to converge for ADI, and G is chosen so that (T,G) is controllable -- in other words, 

### Control Theory

We look at the Sylvester-controller equation

$$TX - XA = -GC $$

The following code will generate an arbitrary $A$ and $B$ to frame the problem, a $T$ constructed based on a random spectrum $\Omega$, and $G$  chosen such that $(T,G)$ is controllable.

In [ ]:
function state_estimation(n)
    A,B,C = gen_balance(n)
    evals = rand(n) .- 2
    T = zeros((n,n))
    T[diagind(T,0)] = evals
    T[diagind(T,-1)] .= 1
    G = zeros(n) 
    G[1] = 1
    return A,T,B,G
end

In [ ]:
A,T,B,G = state_estimation(10)
display("text/plain",A)
display("text/plain",T)
display("text/plain",G)

### PDE

The example below is the 2D heat equation, or

$$u' = \alpha \nabla ^2 u$$

The boundary conditions are randomly generated in the function below. We also ensure that the boundaries of the function $u$ are 0, and use the spectral method to generate dense A and B.

In [6]:
function diffmat(N,k)
    if N == 0 D = []; return end
    if N == 1 D = 0; return end
    D = []
    
    x = chebpts(N);
    w = [.5 ; ones(N-1,1)]; w[2:2:end] .= -1; w[N] = .5*w[N];

    ii = diagind(zeros(N,N))
    Dx = broadcast(-,x,x');       # all pairwise differences
    Dx = Dx + I;            # add identity
    Dxi = 1 ./ Dx;                    # reciprocal 
    Dw = broadcast(/,w',w);    # pairwise divisions
    Dw = Dw - I;            # subtract identity

    D = Dw .* Dxi
    if k == 1
        D = Dw .* Dxi;
        D[ii] .= 0; D[ii] = -1 * sum(D, dims=2);              # negative sum trick
    elseif k == 2 
        D = 2*D .* (repeat(D[ii],1,N) - Dxi);
        D[ii] .= 0; D[ii] = -1 * sum(D, dims=2);              # negative sum trick
    else
        D = k*Dxi .* (Dw.*repeat(D[ii],1,N) - D);
        D[ii] .= 0; D[ii] = -1 * sum(D,dims=2);               # negative sum trick
    end
    
    return D
end

# Chebyshev points on an interval
# https://github.com/ay2718/spectral-pde-solver/blob/master/chebpts.jl
function chebpts(n, interval = [-1.0, 1.0])
    m = n-1;
    chpts = sin.(pi*(-m:2:m) ./ (2*m));
    chpts.*=(0.5*diff(interval));
    chpts.+=(0.5*sum(interval));
    chpts
end

chebpts (generic function with 2 methods)

In [7]:
D = diffmat(20,2)

20×20 Array{Float64,2}:
 -20352.5       21503.8       -1362.5       …     -1.01378        0.5     
   5375.94      -6454.75       1216.68             0.513923      -0.253445
   -340.626      1216.68      -1268.4             -0.535717       0.264117
     68.8393     -175.055       454.395            0.574491      -0.283092
    -22.4913       51.4191      -81.4744          -0.634436       0.3124  
      9.6018      -20.9358       27.7349    …      0.72262       -0.355458
     -4.87197      10.3582      -12.571           -0.850688       0.417877
      2.79354      -5.85079       6.75519          1.03804       -0.508971
     -1.75657       3.64367      -4.07777         -1.318          0.644648
      1.18813      -2.44851       2.68392          1.75034       -0.853258
     -0.853258      1.75034      -1.89107   …     -2.44851        1.18813 
      0.644648     -1.318         1.40924          3.64367       -1.75657 
     -0.508971      1.03804      -1.10145         -5.85079        2.79354 
 

In [8]:
function gen_2dheat(n)
    h = 1 / (n - 1) # resolution of the discretization
    innerA = diffmat(n-2, 2)
    A = [zeros(n-2) innerA zeros(n-2) ]
    A = [zeros(n)'; A ; zeros(n)' ]
    tau = 0.2
    x = repeat([collect(1:1:n)],n)
    x = hcat(x...)
    y = x'
    C = log.(tau .+ abs.(x - y))
    
    return A,C
end

gen_2dheat (generic function with 1 method)

In [120]:
# import .RandAdi: adi_solve

function adi_solve2(A,B,F,N,p,q)
    """
    Uses randomized ADI to solve the equation AX - XB = F
    INPUT:
            A: matrix mxn
            B: matrix mxk
    OUTPUT: X: matrix such that AX = B
    """
    m,m2 = size(A)
    n,n2 = size(B)
    
    @assert m==m2
    @assert n==n2

    sols = []
    Xprev = zeros((m,n))
    for i = 1:N
        display("==================== new iteration")
        Ahalf = (A - p[i] * I)
        Bhalf = Xprev * (B - q[i] * I) + F
        
        Xhalf = RandAdi.rand_matsolve(Ahalf,Bhalf)

        Asolve = (B - q[i] * I)
        Bsolve = (A - q[i] * I) * Xhalf - F
        X = (RandAdi.rand_matsolve(Asolve', Bsolve'))'

        Xprev = X
        display("X")
        display("text/plain",X)
        push!(sols,X)
    end
    return sols
end

adi_solve2 (generic function with 1 method)

In [121]:
A,C = gen_2dheat(6)
# display("text/plain",A)
# display("text/plain",C)

([0.0 0.0 … 0.0 0.0; 0.0 -14.722222222222221 … 0.5 0.0; … ; 0.0 0.5 … -14.722222222222221 0.0; 0.0 0.0 … 0.0 0.0], [-1.6094379124341003 0.1823215567939546 … 1.4350845252893227 1.6486586255873816; 0.1823215567939546 -1.6094379124341003 … 1.1631508098056809 1.4350845252893227; … ; 1.4350845252893227 1.1631508098056809 … -1.6094379124341003 0.1823215567939546; 1.6486586255873816 1.4350845252893227 … 0.1823215567939546 -1.6094379124341003])

In [122]:
include("./randadi.jl")

Main.RandAdi

In [123]:
B = -1 .* A'
N = 3
p = ones(N)
q = ones(N)
sols = adi_solve2(A, B, C, N, p, q )

"==================== new iteration"

"X"

6×6 Adjoint{Float64,Array{Float64,2}}:
 0.0           8.94121e-6   7.16502e-6  …  -9.08857e-6   0.0        
 2.02616e-15   8.79845e-6   7.04617e-6     -8.63488e-6  -1.77636e-15
 7.59111e-6    1.59162e-5   1.41465e-5     -1.42445e-6   7.37303e-6 
 6.3675e-6    -3.02365e-6  -1.13256e-6      1.54118e-5   5.55423e-6 
 1.12745e-6   -4.74829e-6  -3.22e-6         1.09287e-5   3.56163e-6 
 0.0           9.07207e-6   7.29912e-6  …  -8.48345e-6   0.0        

"==================== new iteration"

"X"

6×6 Adjoint{Float64,Array{Float64,2}}:
 0.0          1.73479e-5   1.38373e-5  -1.46175e-5  -1.86647e-5  0.0        
 1.249e-15    7.81596e-7   1.09162e-6   3.14236e-6   3.58419e-6  1.11022e-15
 1.42173e-5   3.24357e-5   2.87289e-5  -6.54958e-7  -4.48524e-6  1.35383e-5 
 1.35953e-5   1.29012e-5   1.30914e-5   1.44002e-5   1.45952e-5  1.28084e-5 
 2.11255e-6  -1.3021e-5   -9.77799e-6   1.6332e-5    1.95342e-5  5.99357e-6 
 0.0          7.33716e-7   5.99482e-7   2.89745e-7   2.52713e-7  0.0        

"==================== new iteration"

"X"

6×6 Adjoint{Float64,Array{Float64,2}}:
 0.0          2.59667e-5   2.07136e-5  -2.17765e-5  -2.7621e-5   0.0       
 3.31601e-6  -7.42207e-6  -5.40359e-6   1.06586e-5   1.30104e-5  2.17268e-6
 2.0834e-5    4.65284e-5   4.13655e-5   1.92443e-7  -5.097e-6    2.1339e-5 
 2.02917e-5   3.06388e-5   2.86495e-5   1.3e-5       1.08672e-5  1.86134e-5
 2.11255e-6  -2.52546e-6  -1.14216e-6   1.21193e-5   1.36316e-5  5.99357e-6
 0.0         -7.43213e-6  -5.87061e-6   7.28783e-6   8.95312e-6  0.0       

3-element Array{Any,1}:
 [0.0 8.941209020728397e-6 … -9.088572580352559e-6 0.0; 2.0261570199409107e-15 8.798452920012282e-6 … -8.634875583459963e-6 -1.7763568394002505e-15; … ; 1.12744515656793e-6 -4.748289004176885e-6 … 1.0928747924331213e-5 3.5616331220400532e-6; 0.0 9.072067565113876e-6 … -8.483448439036437e-6 0.0]   
 [0.0 1.7347856071377342e-5 … -1.8664720486921066e-5 0.0; 1.2490009027033011e-15 7.815957101714445e-7 … 3.5841948972125527e-6 1.1102230246251565e-15; … ; 2.1125523785858746e-6 -1.3021027753822918e-5 … 1.9534224794022548e-5 5.993569045098512e-6; 0.0 7.337160106076649e-7 … 2.5271301802282524e-7 0.0]
 [0.0 2.5966681749723994e-5 … -2.7620950880726226e-5 0.0; 3.3160084126626632e-6 -7.422071387140529e-6 … 1.301035696213266e-5 2.17268442548324e-6; … ; 2.112552379029964e-6 -2.5254618553193734e-6 … 1.3631560615569533e-5 5.9935690424062216e-6; 0.0 -7.432125909047312e-6 … 8.953116283223538e-6 0.0]    

### Image Denoising
For this set of examples, the noisy image is generated from one of the Julia samples. The image can be changed to provide other examples.

$$\hat{F} \Phi_{x}+\sigma_{\eta}^{2} \Phi_{y}^{-1} \hat{F}=G \Phi_{x}$$

We create the distorted image $G$ by adding Gaussian noise. The approximation of the original image is $\hat F$.

In [ ]:
using TestImages
using Images

In [ ]:
function gen_imagedenoise(name)
    rng = MersenneTwister(1234);
    img = testimage(name)
    gimg = Gray.(img)
    mat = convert(Array{Float64}, gimg)
    
    n,m = size(gimg)
    eta = rand(1)[1] 
    noise = zeros((n,m))
    randn!(rng,noise)
    noise *= eta
    
    G = noise + mat
    
    covx = cov(vec(G'), vec(G'))
    covy = cov(vec(G), vec(G))
    coveta = cov(vec(noise), vec(noise))
    
    return G, covx, covy, coveta
end

In [ ]:
G, covx, covy, coveta = gen_imagedenoise("lena_gray_16bit")

In [ ]:
Gray.(G)

References:
- [Linear Algebra and Dynamical Systems](https://web.stanford.edu/class/archive/ee/ee263/ee263.1082/notes/ee263coursereader.pdf) (page 15-8)